<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [ ]:
!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [1]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) > 0, "Not enough GPU hardware devices available"
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

2024-06-28 09:50:26.086290: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 09:50:27.534999: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-28 09:50:27.535245: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-

In [3]:
import sys
sys.path.append("..")

In [4]:
%load_ext autoreload
%autoreload 2

### Reading Data Files:

In [5]:
from mtsa import calculate_aucroc

In [6]:
from mtsa import files_train_test_split
#path_input_1 = '/data/MIMII/fan/id_00/'
X_train, X_test, y_train, y_test = files_train_test_split(path_input_1)
if(len(y_train) == 0): 
    X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
y_test

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

### MFFCMix Model:

In [7]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [7]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.64

### Hitachi Model:

In [5]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

ValueError: Memory growth cannot differ between GPU devices

In [9]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

NameError: name 'model_hitachi' is not defined

### RANSynCoders

In [13]:
from mtsa import RANSynCoders
model_ransyncorders = RANSynCoders(is_acoustic_data=True, mono=True, normal_classifier=1, abnormal_classifier=0, synchronize=True)
model_ransyncorders.fit(X_train, y_train)


Start of frequency pre-train epoch 0
pre-reconstruction_loss: 0.24115552
Start of frequency pre-train epoch 1
pre-reconstruction_loss: 0.23891613
Start of frequency pre-train epoch 2
pre-reconstruction_loss: 0.23675919
Start of frequency pre-train epoch 3
pre-reconstruction_loss: 0.23467274
Start of frequency pre-train epoch 4
found common oscillations at period(s) = [17.38888888888889, 3.5977011494252875, 2.0324675324675328]


2024-06-28 09:53:44.452720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313,20]
	 [[{{node Placeholder/_1}}]]



Start of sine representation pre-train epoch 0
sine_loss: 0.23429495
Start of sine representation pre-train epoch 1
sine_loss: 0.20670985
Start of sine representation pre-train epoch 2
sine_loss: 0.17955303
Start of sine representation pre-train epoch 3
sine_loss: 0.15458995
Start of sine representation pre-train epoch 4
sine_loss: 0.13271163
Start of epoch 0
sine_loss: 0.114641875 upper_bound_loss: 0.10769732 lower_bound_loss: 0.08109015
Start of epoch 1
sine_loss: 0.10061265 upper_bound_loss: 0.10225938 lower_bound_loss: 0.084670566
Start of epoch 2
sine_loss: 0.0909533 upper_bound_loss: 0.093835674 lower_bound_loss: 0.09121064
Start of epoch 3
sine_loss: 0.08561389 upper_bound_loss: 0.08290926 lower_bound_loss: 0.10085592
Start of epoch 4
sine_loss: 0.083941266 upper_bound_loss: 0.070782125 lower_bound_loss: 0.11263214
Start of epoch 5
sine_loss: 0.084315695 upper_bound_loss: 0.059415385 lower_bound_loss: 0.12442606
Start of epoch 6
sine_loss: 0.08543112 upper_bound_loss: 0.0504292

2024-06-28 09:53:45.963129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313,20]
	 [[{{node Placeholder/_1}}]]



Start of epoch 0
sine_loss: 0.08005923 upper_bound_loss: 0.06560284 lower_bound_loss: 0.07506829
Start of epoch 1
sine_loss: 0.07870169 upper_bound_loss: 0.063551046 lower_bound_loss: 0.06592863
Start of epoch 2
sine_loss: 0.0785979 upper_bound_loss: 0.0610605 lower_bound_loss: 0.05940626
Start of epoch 3
sine_loss: 0.07886555 upper_bound_loss: 0.05838856 lower_bound_loss: 0.054800462
Start of epoch 4
sine_loss: 0.07903236 upper_bound_loss: 0.05567413 lower_bound_loss: 0.051494904
Start of epoch 5
sine_loss: 0.07887922 upper_bound_loss: 0.05289309 lower_bound_loss: 0.049029723
Start of epoch 6
sine_loss: 0.078395665 upper_bound_loss: 0.050211072 lower_bound_loss: 0.04716163
Start of epoch 7
sine_loss: 0.07768885 upper_bound_loss: 0.04772815 lower_bound_loss: 0.045934055
Start of epoch 8
sine_loss: 0.07699712 upper_bound_loss: 0.04546256 lower_bound_loss: 0.04529787
Start of epoch 9
sine_loss: 0.07656273 upper_bound_loss: 0.04349941 lower_bound_loss: 0.04510884
Start of sine representa

...

2024-06-28 09:53:48.273133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313,20]
	 [[{{node Placeholder/_1}}]]


sine_loss: 0.05830783 upper_bound_loss: 0.02462751 lower_bound_loss: 0.035807863
Start of epoch 1
sine_loss: 0.057953153 upper_bound_loss: 0.022923725 lower_bound_loss: 0.03341044
Start of epoch 2
sine_loss: 0.05842936 upper_bound_loss: 0.022061724 lower_bound_loss: 0.031772178
Start of epoch 3
sine_loss: 0.058527764 upper_bound_loss: 0.021637505 lower_bound_loss: 0.030576091
Start of epoch 4
sine_loss: 0.058049798 upper_bound_loss: 0.021437686 lower_bound_loss: 0.029890364
Start of epoch 5
sine_loss: 0.057515543 upper_bound_loss: 0.02144333 lower_bound_loss: 0.029897518
Start of epoch 6
sine_loss: 0.057429403 upper_bound_loss: 0.021572886 lower_bound_loss: 0.0303375
Start of epoch 7
sine_loss: 0.05797712 upper_bound_loss: 0.021657875 lower_bound_loss: 0.030821137
Start of epoch 8
sine_loss: 0.058648475 upper_bound_loss: 0.021500753 lower_bound_loss: 0.03106883
Start of epoch 9
sine_loss: 0.058876164 upper_bound_loss: 0.021117605 lower_bound_loss: 0.030926568
Start of sine representati

2024-06-28 09:53:49.413793: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313,20]
	 [[{{node Placeholder/_1}}]]



Start of epoch 0
sine_loss: 0.074112706 upper_bound_loss: 0.04125927 lower_bound_loss: 0.0373981
Start of epoch 1
sine_loss: 0.07369213 upper_bound_loss: 0.038989387 lower_bound_loss: 0.036682803
Start of epoch 2
sine_loss: 0.07382608 upper_bound_loss: 0.036654357 lower_bound_loss: 0.035377108
Start of epoch 3
sine_loss: 0.07382216 upper_bound_loss: 0.03458772 lower_bound_loss: 0.033926718
Start of epoch 4
sine_loss: 0.073488496 upper_bound_loss: 0.03286501 lower_bound_loss: 0.032716956
Start of epoch 5
sine_loss: 0.07314561 upper_bound_loss: 0.0313863 lower_bound_loss: 0.031944096
Start of epoch 6
sine_loss: 0.07301733 upper_bound_loss: 0.030156286 lower_bound_loss: 0.031612694
Start of epoch 7
sine_loss: 0.07305795 upper_bound_loss: 0.02906016 lower_bound_loss: 0.031568196
Start of epoch 8
sine_loss: 0.07327924 upper_bound_loss: 0.028204165 lower_bound_loss: 0.031621788
Start of epoch 9


Pipeline(steps=[('wav2array', Wav2Array(sampling_rate=16000)),
                ('array2mfcc', Array2Mfcc(sampling_rate=16000)),
                ('final_model',
                 <mtsa.models.ransyncoders_components.ransyncoders_base.RANSynCodersBase object at 0x7f55f9363b80>)])

In [14]:
auc = calculate_aucroc(model_ransyncorders, X_test, y_test)
auc

2024-06-28 09:53:58.160140: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313,20]
	 [[{{node Placeholder/_1}}]]
2024-06-28 09:53:58.245603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313,20]
	 [[{{node Placeholder/_1}}]]
2024-06-28 09:53:58.328170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [313

0.27999999999999997